In [2]:
import logging

from dask.distributed import Client
from dask import delayed, compute

client = Client(threads_per_worker=2, n_workers=4,silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 11.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46073,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 11.57 GiB
Comm: tcp://127.0.0.1:37299,Total threads: 2
Dashboard: http://127.0.0.1:40581/status,Memory: 2.89 GiB
Nanny: tcp://127.0.0.1:44589,


In [1]:
param_list ={
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100, 1000],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1, 1],
    'solver': ['saga', 'liblinear'],

}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(param_list)

print(len(list(param_dict)))

def RandomCV(param_list,n=100):
    import random
    return (item for item in random.sample(list(param_list),n))

100


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../dataset/processed_dataset/final_data_S1.csv')
X = df.drop(['class'],axis=1)
y = df['class']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Testing every combination of parameters with train_6 and test_6
@delayed
def run_model(param):
    model_obj = LogisticRegression(**param, max_iter=1000)
    model_obj.fit(x_train, y_train)
    param.update({
        "score":accuracy_score(y_test, model_obj.predict(x_test))
                  })
    return pd.DataFrame(param,index=[0]).iloc[0:1,:]

# create dask parallel function to run xgb
dfs = (run_model(each) for each in param_dict)
value = compute(dfs,num_workers=4)

final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
# not_used = ['n_jobs']
# final.drop(not_used,axis=1,inplace=True)

In [6]:
client.close()

/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/shailesh/Desktop/fuse-project-1/venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did 

In [5]:
final

,C,penalty,solver,tol,score
0,0.1,l1,saga,0.0001,0.750000
1,0.1,l1,saga,0.0010,0.750000
2,0.1,l1,saga,0.0100,0.743421
3,0.1,l1,saga,0.1000,0.743421
4,0.1,l1,saga,1.0000,0.750000
...,...,...,...,...,...
95,1000.0,l2,liblinear,0.0001,0.763158
96,1000.0,l2,liblinear,0.0010,0.763158
97,1000.0,l2,liblinear,0.0100,0.763158
98,1000.0,l2,liblinear,0.1000,0.789474
